In [109]:
import numpy as np
import pandas as pd
from IPython.display import display
from scipy.interpolate import interp1d



In [110]:
# cc_bb_dir = '/mnt/d/physics/gamma-optical/2001/output/bayesian_block_cc/'
cc_bb_dir = '../../2001/output/bayesian_block_cc/'
cc_dir = '../../2001/output/cross_corr/'
cc_sig_dir = '../../2001/output/cross_object_gam+opt/'

In [111]:
object_num = 0

cc_bb_file = cc_bb_dir+'object'+str(object_num).zfill(4)+'_cc_change_points_000.dat'
read_in_bb = np.loadtxt(cc_bb_file)
print(read_in_bb.shape)

bb_time_index = read_in_bb[0].astype(int)
bb_bin_norm = read_in_bb[1]
num_bins = len(bb_time_index)


bb_bin_start = bb_time_index

bb_bin_end = np.zeros(len(bb_bin_start)).astype(np.int)
bb_bin_end[:-1] = bb_time_index[1:]
bb_bin_end[-1] = 14999

print(bb_bin_start)
print(bb_bin_end)


cc_file = cc_dir+'object'+str(object_num).zfill(4)+'_stats000.dat'
# read_in_cc = np.loadtxt(cc_file)
# print(read_in_cc.shape)

cc_sig_file = cc_sig_dir+'cross_object'+str(object_num).zfill(4)+'_all_stats_000.dat'
# read_in_cc_sig = np.loadtxt(cc_sig_file)
# print(read_in_cc_sig.shape)



(2, 170)
[    0  1863  1906  1933  2039  2053  2151  2292  2314  2330  2372  2437
  2531  2617  2654  2715  2800  2867  2910  2965  2994  3065  3078  3118
  3211  3276  3299  3377  3406  3514  3538  3625  3668  3728  3793  3866
  3907  3951  4042  4071  4093  4178  4268  4386  4439  4540  4662  4844
  4934  4959  5059  5108  5183  5273  5386  5742  5770  5848  5904  5947
  5988  6034  6055  6129  6169  6323  6431  6469  6558  6770  6806  6857
  6979  7134  7172  7207  7231  7303  7430  7485  7527  7588  7677  7746
  7808  7881  7964  8135  8188  8227  8398  8587  8662  8762  8847  8870
  8900  8924  8986  9032  9089  9153  9189  9223  9340  9419  9529  9562
  9651  9689  9771  9871 10043 10147 10401 10438 10483 10508 10560 10620
 10676 10918 10970 11031 11081 11206 11331 11481 11560 11622 11652 11680
 11717 11978 12035 12113 12165 12266 12353 12450 12520 12553 12666 12720
 12785 12929 13008 13085 13130 13222 13273 13299 13464 13620 13713 13902
 13972 14037 14076 14102 14146 14177 14205

In [112]:
## Reading in cross correlation values

read_in = np.loadtxt(cc_file)

time_err = read_in[0]
data_err = read_in[1:]

cor_mean = data_err[0]
cor_err_d = cor_mean-data_err[1]
cor_err_u = data_err[2]-cor_mean

# expected_cor_up = cor_err_u *np.sqrt(2/np.pi)
# expected_cor_dn = cor_err_d *np.sqrt(2/np.pi)

# expected_cor = cor_mean + expected_cor_up - expected_cor_dn

# cor_mean = expected_cor

print('cc array shape: '+str(data_err.shape))

cc array shape: (9, 14999)


In [113]:
## Making significance function

read_in = np.loadtxt(cc_sig_file)
time = read_in[0]
data = read_in[1:] ## significance
data = data[:-2] ## getting rid of "all"

cc_significance = np.zeros(len(time))

significance = np.array([0., -1., 1., -2., 2., -3., 3.])
for ii in range(0,len(time)):
    significance_function = interp1d(data.T[ii], significance, axis=0, fill_value='extrapolate')
    cc_significance[ii] = significance_function(cor_mean[ii])

cc_significance[np.isnan(cc_significance)] = 0.
print(cc_significance)

C:\Users\psyko\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:609: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


[0.         0.         0.         ... 0.1925781  0.16455392 0.14711519]


In [114]:
## Find max significance in bb bin

avg_sig = np.zeros(num_bins)
max_sig = np.zeros(num_bins)
max_cc_index = np.zeros(num_bins).astype(np.int)

for ii in range(0,num_bins):
    sig_arr = cc_significance[bb_bin_start[ii]:bb_bin_end[ii]+1]
    cc_arr = cor_mean[bb_bin_start[ii]:bb_bin_end[ii]+1]
    index_arr = np.arange(bb_bin_start[ii], bb_bin_end[ii]+1).astype(np.int)
    if np.average(sig_arr) >= 0:
        avg_sig[ii] = np.average(sig_arr)
        max_sig[ii] = np.amax(sig_arr)
        max_cc_index[ii] = index_arr[np.argmax(cc_arr)] 
    else:
        avg_sig[ii] = np.average(sig_arr)
        max_sig[ii] = np.amin(sig_arr)
        max_cc_index[ii] = index_arr[np.argmin(cc_arr)] 

print(avg_sig[:10])
print(max_sig[:10])
print(max_cc_index[:10])


[-0.00406926 -0.56838118  0.1664509  -0.34715605 -1.19392653 -0.44221404
  0.20154018  0.89682618  1.47846244  0.38465959]
[-0.80232636 -0.89090909  0.68591708 -1.05797101 -1.79954955 -1.38452924
  0.88501742  1.35780564  1.70668906  0.8537859 ]
[   0 1864 1914 1979 2041 2076 2248 2314 2320 2330]


In [115]:
'''
Arrays:
time
cor_mean
bb_bin_start, bb_bin_end, bb_bin_norm
max_sig, max_cc_index

We want 
bin_start_time, bin_end_time, max_cc_in_bin_time, max_cc_in_bin, max_sig_in_bin
for only sig >= +(-) 2 for local max(min)
'''

output_arr = np.zeros([1,5])
for ii in range(1,num_bins-1):
    if time[bb_bin_end[ii]] < -1500. or time[bb_bin_start[ii]] > 2500.:
        continue
    
    if bb_bin_norm[ii] >= 0:
        if bb_bin_norm[ii] > bb_bin_norm[ii+1] and bb_bin_norm[ii] > bb_bin_norm[ii-1]:
            bin_start_time = time[bb_bin_start[ii]]
            bin_end_time = time[bb_bin_end[ii]]
            max_cc_in_bin_time = time[max_cc_index[ii]]
            max_cc_in_bin = cor_mean[max_cc_index[ii]]
            max_sig_in_bin = max_sig[ii]
            avg_sig_in_bin = avg_sig[ii]
            
            if avg_sig_in_bin >= 2.:
                new_row = np.array([bin_start_time, bin_end_time, max_cc_in_bin_time, max_cc_in_bin, max_sig_in_bin])
                output_arr = np.append(output_arr, [new_row], axis=0)
            
    elif bb_bin_norm[ii] < 0:
        if bb_bin_norm[ii] < bb_bin_norm[ii+1] and bb_bin_norm[ii] < bb_bin_norm[ii-1]:
            bin_start_time = time[bb_bin_start[ii]]
            bin_end_time = time[bb_bin_end[ii]]
            max_cc_in_bin_time = time[max_cc_index[ii]]
            max_cc_in_bin = cor_mean[max_cc_index[ii]]
            max_sig_in_bin = max_sig[ii]
            avg_sig_in_bin = avg_sig[ii]
            
            if avg_sig_in_bin <= -2.:
                new_row = np.array([bin_start_time, bin_end_time, max_cc_in_bin_time, max_cc_in_bin, max_sig_in_bin])
                output_arr = np.append(output_arr, [new_row], axis=0)

if len(output_arr) > 1:
    output_arr = output_arr[1:]
    
num_sig_bins = len(output_arr)
output_arr = np.append(object_num*np.ones([num_sig_bins,1]), output_arr, axis = 1)
df_arr = output_arr.T
print(len(output_arr))
print(output_arr)
            

6
[[ 0.00000000e+00 -1.38280000e+03 -1.37120000e+03 -1.37920000e+03
  -4.60150641e-01 -2.53624150e+00]
 [ 0.00000000e+00 -4.70400000e+02 -4.27200000e+02 -4.53200000e+02
   4.51010595e-01  2.34036980e+00]
 [ 0.00000000e+00 -5.60000000e+00  1.12000000e+01 -4.00000000e-01
   6.25290231e-01  3.86681382e+00]
 [ 0.00000000e+00  8.76000000e+02  9.08800000e+02  9.03600000e+02
  -4.40236491e-01 -2.27979641e+00]
 [ 0.00000000e+00  1.01760000e+03  1.05920000e+03  1.04560000e+03
  -5.00919685e-01 -2.80694496e+00]
 [ 0.00000000e+00  2.17200000e+03  2.20360000e+03  2.18440000e+03
   5.00423477e-01  2.73611219e+00]]


In [123]:
'''
Convert data to pandas data frame
'''
df_dict = {'object_num':df_arr[0].astype(np.int), 
           'time_start':df_arr[1], 
           'time_end':df_arr[2], 
           'max_cc_time':df_arr[3], 
           'max_cc_val':df_arr[4], 
           'max_sig':df_arr[4]}

df = pd.DataFrame(df_dict)
df = df.sort_values(by=['object_num', 'max_cc_val'], ascending=False)
# print(df.to_markdown)
display(df)



,object_num,time_start,time_end,max_cc_time,max_cc_val,max_sig
2,0,-5.6,11.2,-0.4,0.625290,0.625290
5,0,2172.0,2203.6,2184.4,0.500423,0.500423
1,0,-470.4,-427.2,-453.2,0.451011,0.451011
3,0,876.0,908.8,903.6,-0.440236,-0.440236
0,0,-1382.8,-1371.2,-1379.2,-0.460151,-0.460151
4,0,1017.6,1059.2,1045.6,-0.500920,-0.500920
